### This is the jupyter notebook for sentiment analysis of the tweets using the Twitter API
### FYP project
### Author: Muhammad Abu Bakar
### Author: Muhammad Faiz Hassan


In [68]:
# 1. Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tweepy
from tweepy import OAuthHandler


In [69]:


# configured api's with consumer k

# Use the above credentials to authenticate the API.

auth = tweepy.OAuthHandler( "gxoci9hQ7adNawtYj4r5J459y" , "Vn1HMAVwH7Bl3MZXlwGA5kaCOFvs9ZOJ4b3Wg1JRSapU2kbA4G" )
auth.set_access_token( "930155901666971648-lrpZsfrmdQQihbTLUZKs6pASEdNS5ek" , "sjiascsBPd8UBkqPNPUYbaL0RoLxnD9gikEmmRGEusftr" )
api = tweepy.API(auth)
print(api)

In [70]:
# 3.To get the tweets in a Proper format, first lets create a Dataframe to store the extracted data.

df = pd.DataFrame(columns=["date","User","IsVerified","Tweet","Likes","RT",'User_location'])

# df['date'] = df['date'].apply(lambda a: pd.to_datetime(a).date())
# df['date'] = pd.to_datetime(df['date'])
# df.dtypes
# df['date'] = df['date'].dt.tz_localize(None)
print(df)



Empty DataFrame
Columns: [date, User, IsVerified, Tweet, Likes, RT, User_location]
Index: []


## We will use api as api.search inside this tweepy cursor.


In [71]:
# 4.Write a Function to extract tweets:

# We will Use **tweepy.cursor()** because we want to extract a larger number of tweets i.e over 100,500 etc


def get_tweets(Topic,Count):    
    i=0
    for tweet in tweepy.Cursor(api.search_tweets, q=Topic,count=10, lang="en",exclude='retweets').items():
        print(i, end='\r')
        date=tweet.created_at
        df.loc[i,"date"] = date.strftime("%Y-%m-%d")
        # print('hello')
        df.loc[i,"User"] = tweet.user.name
        df.loc[i,"IsVerified"] = tweet.user.verified
        df.loc[i,"Tweet"] = tweet.text
        df.loc[i,"Likes"] = tweet.favorite_count
        df.loc[i,"RT"] = tweet.retweet_count
        df.loc[i,"User_location"] = tweet.user.location
        #df.to_csv("TweetDataset.csv",index=False)
        df.to_excel('{}.xlsx'.format("TweetDataset"),index=False)   ## Save as Excel
        i=i+1
        if i>Count:
            break
        else:
            pass

# take input from user
Topic = "Imran Khan"
# Topic=["Pakistan"]
get_tweets(Topic , Count=10)

Unexpected parameter: exclude
Unexpected parameter: exclude


In [72]:
# Showing the data the Data
df.head(8)

,date,User,IsVerified,Tweet,Likes,RT,User_location
0,2022-11-26,Arslaan,False,@ImranRiazKhan Great leader Imran Khan 🫡🫡🫡🫡,0,0,
1,2022-11-26,Neo News Urdu,True,🔴 LIVE | Huge Crowd at PTI Power Show In Rawal...,1,0,Pakistan
2,2022-11-26,FightAgainstDisinformation,False,Imran Khan: Fist rally by former Pakistan PM s...,0,0,London
3,2022-11-26,ANGRY MAN,False,PRAYERS FOR IMRAN KHAN &amp; PEACEFUL LONGMARC...,0,1,"Karachi, Pakistan"
4,2022-11-26,Ali Sadam❤️ AS ❤️ PTI 💕,False,@RashidNasrulah This leader Imran Khan Sahab g...,1,0,
5,2022-11-26,Ayesha,False,"Slow internet, because we are not scared of Im...",0,0,
6,2022-11-26,Ehsan Ghuman,False,Municipal elections are going to be held in Az...,0,0,
7,2022-11-26,Bis Faqiri,False,@ShaziaJ3 @MaryamNSharif Imran khan is like ra...,0,0,


In [73]:
# Function to Clean the Tweet.

import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|([RT])', ' ', str(tweet).lower()).split())

# We only want the Text so :

# (@[A-Za-z0-9]+)   : Delete Anything like @hello @Letsupgrade etc
# ([^0-9A-Za-z \t]) : Delete everything other than text,number,space,tabspace
# (\w+:\/\/\S+)     : Delete https://
# ([RT]) : Remove "RT" from the tweet

In [74]:
from textblob import TextBlob
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [75]:
#Function to Pre-process data for Worlcloud:here we are removing the words present in Topic from the Corpus so they dont come in WordCloud.
# Ex : Topic is "Arsenal vs United", we want to remove "Arsenal" "vs" "United" from the WordCloud.

def prepCloud(Topic_text,Topic):
    Topic = str(Topic).lower()
    Topic=' '.join(re.sub('([^0-9A-Za-z \t])', ' ', Topic).split())
    Topic = re.split("\s+",str(Topic))
    stopwords = set(STOPWORDS)
    stopwords.update(Topic) ### Add our topic in Stopwords, so it doesnt appear in wordClous
    ###
    text_new = " ".join([txt for txt in Topic_text.split() if txt not in stopwords])
    return text_new


In [76]:
df['clean_tweet'] = df['Tweet'].apply(lambda x : clean_tweet(x))
df.head(5)


,date,User,IsVerified,Tweet,Likes,RT,User_location,clean_tweet
0,2022-11-26,Arslaan,False,@ImranRiazKhan Great leader Imran Khan 🫡🫡🫡🫡,0,0,,great leader imran khan
1,2022-11-26,Neo News Urdu,True,🔴 LIVE | Huge Crowd at PTI Power Show In Rawal...,1,0,Pakistan,live huge crowd at pti power show in rawalpind...
2,2022-11-26,FightAgainstDisinformation,False,Imran Khan: Fist rally by former Pakistan PM s...,0,0,London,imran khan fist rally by former pakistan pm si...
3,2022-11-26,ANGRY MAN,False,PRAYERS FOR IMRAN KHAN &amp; PEACEFUL LONGMARC...,0,1,"Karachi, Pakistan",prayers for imran khan amp peaceful longmarchi...
4,2022-11-26,Ali Sadam❤️ AS ❤️ PTI 💕,False,@RashidNasrulah This leader Imran Khan Sahab g...,1,0,,this leader imran khan sahab great


In [77]:
df["Sentiment"] = df["Tweet"].apply(lambda x : analyze_sentiment(x))
df.head(5)

,date,User,IsVerified,Tweet,Likes,RT,User_location,clean_tweet,Sentiment
0,2022-11-26,Arslaan,False,@ImranRiazKhan Great leader Imran Khan 🫡🫡🫡🫡,0,0,,great leader imran khan,positive
1,2022-11-26,Neo News Urdu,True,🔴 LIVE | Huge Crowd at PTI Power Show In Rawal...,1,0,Pakistan,live huge crowd at pti power show in rawalpind...,positive
2,2022-11-26,FightAgainstDisinformation,False,Imran Khan: Fist rally by former Pakistan PM s...,0,0,London,imran khan fist rally by former pakistan pm si...,Neutral
3,2022-11-26,ANGRY MAN,False,PRAYERS FOR IMRAN KHAN &amp; PEACEFUL LONGMARC...,0,1,"Karachi, Pakistan",prayers for imran khan amp peaceful longmarchi...,positive
4,2022-11-26,Ali Sadam❤️ AS ❤️ PTI 💕,False,@RashidNasrulah This leader Imran Khan Sahab g...,1,0,,this leader imran khan sahab great,positive


In [78]:


# Check Summary of Random Record
n = 5
print("Original tweet:\n",df['Tweet'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])



Original tweet:
 Slow internet, because we are not scared of Imran Khan. 
#PTILongMarch https://t.co/lMLlOlKzea

Clean tweet:
 slow internet because we are not scared of imran khan ptilongmarch

Sentiment of the tweet:
 Negative


In [79]:
# Overall Summary

print("Total Tweets Extracted for Topic : {} are : {}".format(Topic,len(df.Tweet)))
print("Total Positive Tweets are : {}".format(len(df[df["Sentiment"]=="Positive"])))
print("Total Negative Tweets are : {}".format(len(df[df["Sentiment"]=="Negative"])))
print("Total Neutral Tweets are : {}".format(len(df[df["Sentiment"]=="Neutral"])))


Total Tweets Extracted for Topic : Imran Khan are : 11
Total Positive Tweets are : 0
Total Negative Tweets are : 1
Total Neutral Tweets are : 6


In [80]:
df["Sentiment"].value_counts()

Neutral     6
positive    4
Negative    1
Name: Sentiment, dtype: int64

In [81]:


#sns.countplot(df["Sentiment"],facecolor=(0, 0, 0, 0),linewidth=5,edgecolor=sns.color_palette("dark", 3))
sns.countplot(df["Sentiment"])
plt.title("Summary of Counts for Total tweets")



ValueError: could not convert string to float: 'positive'

In [ ]:


# Piechart 
#string, used to label the wedges with their numeric value. The label will be placed inside the wedge. The format string will be fmt%pct.

a=len(df[df["Sentiment"]=="Positive"])
b=len(df[df["Sentiment"]=="Negative"])
c=len(df[df["Sentiment"]=="Neutral"])
d=np.array([a,b,c])
explode = (0.1, 0.0, 0.1)
plt.pie(d,shadow=True,explode=explode,labels=["Positive","Negative","Neutral"],autopct='%1.2f%%');



In [82]:
#check if verified user has more tweets

sns.countplot(df["Sentiment"],hue=df.IsVerified)
plt.title("Summary of Counts for Total tweets,Distributed by if the User has a verified Account or not")



ValueError: Cannot use `hue` without `x` and `y`